In [2]:
import json
import pandas as pd

### 1. check the contents in some files and seperate the training and testing data

#### (1) observe data_identification.csv and seperate train and test identifications

In [2]:
#data_identification.csv
train_test_set = pd.read_csv("dm19-lab2-nthu/data_identification.csv",
                         header=None, names=["id", "identification"])

In [3]:
train_test_set.head()

,id,identification
0,tweet_id,identification
1,0x28cc61,test
2,0x29e452,train
3,0x2b3819,train
4,0x2db41f,test


In [4]:
#drop the first title row
train_test_set = train_test_set.drop([0], axis=0)

In [5]:
train_test_set.head()

,id,identification
1,0x28cc61,test
2,0x29e452,train
3,0x2b3819,train
4,0x2db41f,test
5,0x2a2acc,train


In [6]:
train_test_set[:10]

,id,identification
1,0x28cc61,test
2,0x29e452,train
3,0x2b3819,train
4,0x2db41f,test
5,0x2a2acc,train
6,0x2a8830,train
7,0x20b21d,train
8,0x2452cf,train
9,0x2d729d,train
10,0x2ab56d,train


In [50]:
train_ids = train_test_set[train_test_set['identification']=='train'].filter(['id'])

In [51]:
test_ids = train_test_set[train_test_set['identification']=='test'].filter(['id'])

In [52]:
train_ids[:10]

,id
2,0x29e452
3,0x2b3819
5,0x2a2acc
6,0x2a8830
7,0x20b21d
8,0x2452cf
9,0x2d729d
10,0x2ab56d
11,0x1f3657
12,0x1fcc53


P.S. total number of tweet_id is 1867535

In [57]:
len(train_ids)

1455563

In [58]:
len(test_ids)

411972

P.S. emotion.csv only has train_ids

#### (2) observe emotion.csv

In [22]:
#emotion.csv
emotion_set = pd.read_csv("dm19-lab2-nthu/emotion.csv",
                         header=None, names=["id", "emotion"])

In [23]:
#drop the first title row
emotion_set = emotion_set.drop([0], axis=0)

In [24]:
emotion_set[:10]

,id,emotion
1,0x3140b1,sadness
2,0x368b73,disgust
3,0x296183,anticipation
4,0x2bd6e1,joy
5,0x2ee1dd,anticipation
6,0x34cd80,joy
7,0x33f099,sadness
8,0x2ae7b7,sadness
9,0x2408d4,trust
10,0x2b193b,sadness


In [25]:
len(emotion_set)

1455563

In [26]:
## save to pickle file
emotion_set.to_pickle("train_df.pkl") 

P.S. tweets_DM.json includes both train and test data

### 2. data preprocessing

#### observe tweets_DM.json and get texts and tweets_id from raw data

In [27]:
#tweets_DM.json
tweets_df = pd.read_json("dm19-lab2-nthu/tweets_DM.json", lines=True)

In [28]:
tweets_df.head()

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets


In [30]:
tweets_source_df = tweets_df['_source']

In [31]:
tweets_df['text'] = tweets_source_df.apply(lambda x: x['tweet']['text'])

In [32]:
tweets_df['id'] = tweets_df['_source'].apply(lambda x: x['tweet']['tweet_id'])

In [33]:
filter_tweets_df = tweets_df.filter(['text','id'])

In [34]:
filter_tweets_df.head()

,text,id
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350
2,"Confident of your obedience, I write to you, k...",0x28b412
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
4,"""Trust is not the same as faith. A friend is s...",0x2de201


In [53]:
train_tweets_df = filter_tweets_df[filter_tweets_df['id'].isin(train_ids['id'])]

In [55]:
test_tweets_df = filter_tweets_df[~filter_tweets_df['id'].isin(train_ids['id'])]

In [54]:
len(train_tweets_df)

1455563

In [56]:
len(test_tweets_df)

411972

In [59]:
train_tweets_df.head()

,text,id
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0
5,@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c
6,Still waiting on those supplies Liscus. <LH>,0x2c91a8


In [60]:
test_tweets_df.head()

,text,id
2,"Confident of your obedience, I write to you, k...",0x28b412
4,"""Trust is not the same as faith. A friend is s...",0x2de201
9,When do you have enough ? When are you satisfi...,0x218443
30,"God woke you up, now chase the day #GodsPlan #...",0x2939d5
33,"In these tough times, who do YOU turn to as yo...",0x26289a


In [62]:
## save to pickle file
train_tweets_df.to_pickle("train_tweets_df.pkl") 
test_tweets_df.to_pickle("test_tweets_df.pkl")

### 3. using a pre-trained w2v model

#### (1) load GoogleNews vectors

In [63]:
from gensim.models import KeyedVectors
from gensim.utils import tokenize
from gensim.models import Word2Vec, Phrases

## Note: this model is very huge, this will take some time ...
model_path = "GoogleNews/GoogleNews-vectors-negative300.bin.gz"
w2v_google_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
print('load ok')

c:\users\grace\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


load ok


#### (2) load train and test data

In [64]:
train_tweets_df = pd.read_pickle("train_tweets_df.pkl")
test_tweets_df = pd.read_pickle("test_tweets_df.pkl")

#### (3) tokenize and use w2v

In [65]:
import nltk
import numpy as np

In [66]:
def tokenize_text(sentence):
    tokens = nltk.wordpunct_tokenize(sentence.lower())	# 請注意，差異在cut-off
    return tokens

In [68]:
# tokenize texts using tokenize_text()
train_tweets_df['unigrams'] = train_tweets_df['text'].apply(lambda x: tokenize_text(x))

In [69]:
# tokenize texts using tokenize_text()
test_tweets_df['unigrams'] = test_tweets_df['text'].apply(lambda x: tokenize_text(x))

In [78]:
train_tweets_df.head()

,text,id,unigrams
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20,"[people, who, post, "", add, me, on, #, snapcha..."
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350,"[@, brianklaas, as, we, see, ,, trump, is, dan..."
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,"[now, issa, is, stalking, tasha, 😂😂😂, <, lh, >]"
5,@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,"[@, riskshow, @, thekevinallison, thx, for, th..."
6,Still waiting on those supplies Liscus. <LH>,0x2c91a8,"[still, waiting, on, those, supplies, liscus, ..."


In [70]:
#save w2v index2word to a set
index2word_set = set(w2v_google_model.wv.index2word)

In [74]:
def w2v(tokens):
    num_of_word = 0
    sentence_vec = np.zeros(300)
    avg_vec = np.zeros(300)
    for word in tokens:
        if word not in index2word_set:
            continue
        else:
            sentence_vec = np.add(sentence_vec,w2v_google_model[word])
            num_of_word += 1
        if(num_of_word>0):
            length = np.linalg.norm(sentence_vec)
            avg_vec = sentence_vec/length
    return avg_vec

In [79]:
# translate unigrams to vector using w2v()
train_tweets_df['vector'] = train_tweets_df['unigrams'].apply(lambda x: w2v(x))

In [80]:
test_tweets_df['vector'] = test_tweets_df['unigrams'].apply(lambda x: w2v(x))

In [81]:
train_tweets_df.head()

,text,id,unigrams,vector
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20,"[people, who, post, "", add, me, on, #, snapcha...","[-0.04398071877257593, 0.005973617156422666, 0..."
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350,"[@, brianklaas, as, we, see, ,, trump, is, dan...","[-0.07402511888368858, -0.027571952599088873, ..."
3,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,"[now, issa, is, stalking, tasha, 😂😂😂, <, lh, >]","[-0.08825499574715533, 0.021097196412366435, 0..."
5,@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,"[@, riskshow, @, thekevinallison, thx, for, th...","[-0.05885132318051363, -0.02492928390044968, 0..."
6,Still waiting on those supplies Liscus. <LH>,0x2c91a8,"[still, waiting, on, those, supplies, liscus, ...","[-0.01618322244071664, -0.02359793926091037, 0..."


In [83]:
#add emotion labels to train dataframe
train_tweets_df = train_tweets_df.merge(emotion_set)

In [84]:
train_tweets_df.head()

,text,id,unigrams,vector,emotion
0,"People who post ""add me on #Snapchat"" must be ...",0x376b20,"[people, who, post, "", add, me, on, #, snapcha...","[-0.04398071877257593, 0.005973617156422666, 0...",anticipation
1,"@brianklaas As we see, Trump is dangerous to #...",0x2d5350,"[@, brianklaas, as, we, see, ,, trump, is, dan...","[-0.07402511888368858, -0.027571952599088873, ...",sadness
2,Now ISSA is stalking Tasha 😂😂😂 <LH>,0x1cd5b0,"[now, issa, is, stalking, tasha, 😂😂😂, <, lh, >]","[-0.08825499574715533, 0.021097196412366435, 0...",fear
3,@RISKshow @TheKevinAllison Thx for the BEST TI...,0x1d755c,"[@, riskshow, @, thekevinallison, thx, for, th...","[-0.05885132318051363, -0.02492928390044968, 0...",joy
4,Still waiting on those supplies Liscus. <LH>,0x2c91a8,"[still, waiting, on, those, supplies, liscus, ...","[-0.01618322244071664, -0.02359793926091037, 0...",anticipation


In [86]:
test_tweets_df.head()

,text,id,unigrams,vector
2,"Confident of your obedience, I write to you, k...",0x28b412,"[confident, of, your, obedience, ,, i, write, ...","[-0.013770567964291183, -0.03728248351083156, ..."
4,"""Trust is not the same as faith. A friend is s...",0x2de201,"["", trust, is, not, the, same, as, faith, ., a...","[0.030071268925277977, -0.036261832915662436, ..."
9,When do you have enough ? When are you satisfi...,0x218443,"[when, do, you, have, enough, ?, when, are, yo...","[0.0605162729192968, -0.0009634782860012661, 0..."
30,"God woke you up, now chase the day #GodsPlan #...",0x2939d5,"[god, woke, you, up, ,, now, chase, the, day, ...","[0.01819009581685019, 0.001476407360784927, 0...."
33,"In these tough times, who do YOU turn to as yo...",0x26289a,"[in, these, tough, times, ,, who, do, you, tur...","[0.017859831789070642, 0.09871569549570929, 0...."


In [85]:
#filter useful columns
train_tweets_df = train_tweets_df.filter(['id','vector','emotion'])

In [87]:
test_tweets_df = test_tweets_df.filter(['id','vector'])

In [88]:
#drop nan value in train and replace nan value with all zero vector in test
train_tweets_df = train_tweets_df.dropna()

In [89]:
len(train_tweets_df)

1455563

#### (4) train a Naive Bayes model and predict the testing data

In [15]:
import numpy as np

In [3]:
#We couldn't save train_tweets_vec_df.pkl by jupyter since the kernal will be crashed, We save the files in local environment
train_tweets_df = pd.read_pickle("train_tweets_vec_df.pkl")

In [13]:
#prepare X and Y training data and label
X_train = train_tweets_df['vector'][:int(len(train_tweets_df)*3/4)]
Y_train = train_tweets_df['emotion'][:int(len(train_tweets_df)*3/4)]

In [16]:
X_train = np.array(X_train)

In [17]:
len(X_train)

1091672

In [18]:
len(Y_train)

1091672

In [19]:
#also prepare X testing data
X_test = train_tweets_df['vector'][int(len(train_tweets_df)*3/4):]
Y_test = train_tweets_df['emotion'][int(len(train_tweets_df)*3/4):]

In [20]:
X_test = np.array(X_test)

In [21]:
from sklearn.svm import LinearSVC

In [ ]:
## build LinearSVC model
LSVC_model = LinearSVC()

## training!
LSVC_model = LSVC_model.fit(list(X_train), Y_train)

## predict!
y_train_pred = LSVC_model.predict(list(X_train))
y_test_pred = LSVC_model.predict(list(X_test))

In [ ]:
with open('LSVC_clf.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [ ]:
## accuracy
from sklearn.metrics import accuracy_score
acc_train = accuracy_score(y_true=Y_train,y_pred=y_train_pred)
acc_test = accuracy_score(y_true=Y_test,y_pred=y_test_pred)
print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

#### (5) write submission file

In [ ]:
test_tweets_df = pd.read_pickle("test_tweets_vec_df.pkl")

In [ ]:
with open('LSVC_clf.pickle', 'rb') as f:
    LSVC_model = pickle.load(f)

In [ ]:
X_unknown = test_tweets_df['vector']
X_unknown = np.array(X_unknown)
y_unknown_pred = LSVC_model.predict(list(X_unknown))

In [ ]:
predict_test = {
    "id":list(test_tweets_df['id']),
    "emotion":list(y_unknown_pred)
}

In [ ]:
predict_test = pd.DataFrame(predict_test)
predict_test.to_csv('result.csv')